### Leer Archivos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -r /content/drive/MyDrive/Recomendations_Yelp_GMap/requeriments.txt

In [ ]:
import pandas as pd
import geopandas as gpd
import fiona
import numpy as np
import json

# Nueva sección

In [ ]:

business_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/RAW_Data/Yelp_Data/business.pkl'
# Carga del archivo bussines.pkl en formato pickle
df_business = pd.read_pickle(business_file_path)



In [ ]:

# SEPARAR LAS COLUMNAS DUPLICADAS SIN VALORES Y CON NOMBRES REPETIDOS
df_business = df_business.iloc[:, :14]
# MEDIANTE EL ARCHIVO NOMBRADO SE OBTUVIERON LOS VALORES EN LATITUD Y LONGITUD DE LA SUPERFICIE
#ESPECIFICA DE CADA ESTADO DE USA.
#Los nombres de los estados estan escritos en minuscula y español para simplificar el codigo
dic_states =  {
                'alabama': 'AL',
                'alaska': 'AK',
                'arkansas' : 'AR' ,
                'california' : 'CA',
                'colorado' : 'CO',
                'connecticut' : 'CT',
                'delaware' : 'DE' ,
                'florida' : 'FL' ,
                'hawaii' : 'HI' ,
                'idaho' : 'ID' ,
                'illinois' : 'IL' ,
                'iowa' : 'IA' ,
                'kansas' : 'KS' ,
                'luisiana' : 'LA' ,
                'maine' : 'ME' ,
                'maryland' : 'MD' ,
                'massachusetts' : 'MA' ,
                'minnesota' : 'MN' ,
                'misisipi' : 'MS'           

            }
#Simulamos que el usario introduce el nombre del estado de la federación: Probar con Luisiana y California
estado_federal = 'California'
estado_federal = estado_federal.lower()
inicial_estado = dic_states[estado_federal]
shapefile_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/RAW_Data/Yelp_Data/cb_2018_us_state_500k.shp'
gdf = gpd.read_file(shapefile_path)
gdf_st = gdf[gdf['STUSPS'] == inicial_estado]


#MEDIANTE ESTE PASO SE REALIZA EL FILTRADO DE LOS DATOS SEGUN SU UBICACION EN LATITUD Y LONGITUD, DEFINIDA PREVIAMENTE. 
#


gdf_st = gpd.read_file(shapefile_path)
st_polygon = gdf_st[gdf_st['STUSPS'] == inicial_estado]['geometry'].iloc[0]

# Carga los datos en un DataFrame de Pandas
df_filtered = gpd.GeoDataFrame(df_business, geometry=gpd.points_from_xy(df_business.longitude, df_business.latitude))
df_filtered = df_filtered[df_filtered.geometry.within(st_polygon)]

#ESTOS SON LOS ESTABLECIMIENTOS DEL ESTADO SELECCIONADO DISPONIBLES EN LA BASE DE DATOS
#SE CORROBORÓ QUE LA BASE DE DATOS NO ES RESPRESENTATIVA DE LA REALIDAD, PUES EN CIUDADES MUY POBLADAS NO ENCONTRAMOS DATOS. 

df_filtered.info()
# RESULTADO ESPERADO:  FILAS


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 5175 entries, 0 to 150303
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   business_id   5175 non-null   object  
 1   name          5175 non-null   object  
 2   address       5175 non-null   object  
 3   city          5175 non-null   object  
 4   state         5174 non-null   object  
 5   postal_code   5175 non-null   object  
 6   latitude      5175 non-null   object  
 7   longitude     5175 non-null   object  
 8   stars         5175 non-null   object  
 9   review_count  5175 non-null   object  
 10  is_open       5175 non-null   object  
 11  attributes    4642 non-null   object  
 12  categories    5174 non-null   object  
 13  hours         4282 non-null   object  
 14  geometry      5175 non-null   geometry
dtypes: geometry(1), object(14)
memory usage: 646.9+ KB


In [ ]:
#SE FILTRA POR TODOS LOS LOCALES QUE CONTENGAN EN ALGUNA CATEGORIA RESTAURANT COMO CATEGORIA
df_restaurants = df_filtered[df_filtered['categories'].str.contains('Restaurant|restaurant', na=False)]
df_restaurants.info()
#RESULTADO ESPERADO: 1160 FILAS. 
restaurants_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/Clean_Data/'+'Y_bussines_'+inicial_estado+'.parquet'
df_restaurants.to_parquet(restaurants_file_path)


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1160 entries, 85 to 150290
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   business_id   1160 non-null   object  
 1   name          1160 non-null   object  
 2   address       1160 non-null   object  
 3   city          1160 non-null   object  
 4   state         1160 non-null   object  
 5   postal_code   1160 non-null   object  
 6   latitude      1160 non-null   object  
 7   longitude     1160 non-null   object  
 8   stars         1160 non-null   object  
 9   review_count  1160 non-null   object  
 10  is_open       1160 non-null   object  
 11  attributes    1150 non-null   object  
 12  categories    1160 non-null   object  
 13  hours         987 non-null    object  
 14  geometry      1160 non-null   geometry
dtypes: geometry(1), object(14)
memory usage: 145.0+ KB


### Lectura del archivo tips

In [ ]:
# Especifica el archivo tips.json y el número de líneas a leer en cada iteración

tip_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/RAW_Data/Yelp_Data/tip.json'
# Fijamos el tamaño de la partición para cargar el archivo por partes en memoria
chunksize = 100000
# Inicializa un objeto vacío de pandas para almacenar los datos
df_tip = pd.DataFrame()

# Lee el archivo JSON en partes y agrega los datos al DataFrame
for chunk in pd.read_json(tip_file_path, lines=True, chunksize=chunksize):
    df_tip = pd.concat([df_tip, chunk], ignore_index=True)
    
df_tip.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


In [ ]:
#SE FILTRA Y UNE LOS TIPS AL ARCHIVO CON LOS RESTAURANTES DEL ESTADO SELECCIONADO
restaurants_tip_table = pd.merge(df_restaurants, df_tip, on='business_id')
restaurants_tip_table = restaurants_tip_table.iloc[:, np.r_[0, 15:restaurants_tip_table.shape[1]]]
restaurants_tip_table.drop_duplicates(inplace=True)
restaurants_tips_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/Clean_Data/'+'Y_tips_'+inicial_estado+'.parquet'
restaurants_tip_table.to_parquet(restaurants_tips_file_path)

restaurants_tip_table.info()

<ipython-input-12-128ecfd77368>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_tip_table.drop_duplicates(inplace=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 77373 entries, 0 to 77375
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   business_id       77373 non-null  object        
 1   user_id           77373 non-null  object        
 2   text              77373 non-null  object        
 3   date              77373 non-null  datetime64[ns]
 4   compliment_count  77373 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 3.5+ MB


### Lectura archivo user.parquet

In [ ]:
#LECTURA

user_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/RAW_Data/Yelp_Data/user.parquet'

df_users = pd.read_parquet(user_file_path)
#SE FILTRA EL ARRCHIVO A SOLO LOS USUARIOS QUE HAN REALIZADO TIPS EN LOCALES DEL ESTADO SELECCIONADO. 
df_state_users = pd.merge( restaurants_tip_table, df_users, on='user_id', how='inner')
df_state_users = df_state_users.iloc[:, [2] + list(range(9, len(df_state_users.columns)))]
df_state_users.drop_duplicates(inplace=True)

# Guardamos el archivo de usuarios del estado seleccionado
st_user_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/Clean_Data/'+'Y_user_'+inicial_estado+'.parquet'
df_state_users.to_parquet(st_user_file_path)

### Lectura del archivo checking.json

In [ ]:
checkin_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/RAW_Data/Yelp_Data/checkin.json'
# Abre el archivo JSON y lee línea por línea
with open(checkin_file_path, 'r') as f:
    data = []
    for line in f:
        # Convierte la línea JSON en un diccionario de Python
        d = json.loads(line)
        data.append(d)

# Crea un DataFrame a partir de la lista de diccionarios
df_checkin = pd.DataFrame(data)

### Lectura del archivo review

In [ ]:
# Especifica el archivo JSON y el número de líneas a leer en cada iteración

review_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/RAW_Data/Yelp_Data/review.json'
# Fijamos el tamaño de las particiones para leer el archivo por partes
chunksize = 1000000

# Inicializa un objeto vacío de pandas para almacenar los datos
df_reviews = pd.DataFrame()

# Lee el archivo JSON en partes y agrega los datos al DataFrame
for chunk in pd.read_json(review_file_path, lines=True, chunksize=chunksize):
    df_reviews = pd.concat([df_reviews, chunk], ignore_index=True)
    
df_reviews.info()

In [ ]:
#SE FILTRA Y UNE AL ARCHIVO CON LOS RESTAURANTES DEL ESTADO SELECCIONADO
restaurants_reviews_table = pd.merge(df_restaurants, df_reviews, on='business_id')
restaurants_reviews_table = restaurants_reviews_table.iloc[:, np.r_[0, 15:restaurants_reviews_table.shape[1]]]

# Guardamos el archivo de restorantes y review del estado seleccionado
st_review_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/Clean_Data/'+'Y_review_'+inicial_estado+'.parquet'
restaurants_reviews_table.to_parquet(st_review_file_path)

restaurants_reviews_table.info()

NameError: ignored

In [ ]:
#SE CREA UN ARCHIVO ESPECIFICO PARA TRABAJAR EL MODELO DE RECOMENDACION.  
df_rating = restaurants_reviews_table.iloc[:, [0, 2, 3]]
rating_file_path = '/content/drive/MyDrive/Recomendations_Yelp_GMap/Clean_Data/'+'Y_rating_'+inicial_estado+'.parquet'
df_rating.to_parquet(rating_file_path)


NameError: ignored